<a href="https://colab.research.google.com/github/Amartyaveer/ChatterBotter_Task/blob/master/Program1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extracting noun form fetch_20newsgroups and storing data to Mongodb

In [1]:
!pip install pymongo[srv]
import pymongo
from pymongo import MongoClient
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
from nltk.stem.porter import *
from bs4 import BeautifulSoup

In [4]:
from sklearn.datasets import fetch_20newsgroups

fetch_20newsgroups = fetch_20newsgroups(remove=('headers', 'footers', 'quotes'))

In [6]:
fetch_20newsgroups.keys()

dict_keys(['data', 'filenames', 'target_names', 'target', 'DESCR'])

In [7]:
data = pd.DataFrame([fetch_20newsgroups.data,[fetch_20newsgroups.target]]).T

In [8]:
data = data.drop(1, axis=1)

In [9]:
text = ""
for i in data[0]:
    for j in i.split(" "):
            text+=j+" "

In [14]:
def review_to_words(review):
    nltk.download(["stopwords", "wordnet"], quiet=True)
    
    text = BeautifulSoup(review, "html.parser").get_text() # Remove HTML tags
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower()) # Convert to lower case
    words = text.split() # Split string into words
    noun_list = [i.name().split('.', 1)[0] for i in wn.all_synsets('n')]
    words = [w for w in words if w  in noun_list] # selects noun

    return words
    

In [11]:
words = review_to_words(text)

In [43]:
noun_words=set()
for i in words:
  if(len(i) > 3):
    noun_words.add(i)
  


In [47]:
noun_word_dict = {}

for i in range(len(noun_words)):
    noun_word_dict[i] = (list(noun_words)[i])

In [48]:
cluster = MongoClient("mongodb+srv://Amartyaveer72:<password>@cluster0.qjtjq.mongodb.net/fetch_20newsgroups?retryWrites=true&w=majority")

In [49]:
db = cluster['fetch_20newsgroups']
collections = db['Noun']

In [50]:
for key, val in noun_word_dict.items():
  collections.insert_one({"_id": key, "name": val})